In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('cnaes - DESAFIO M2 IGTI DeCloud') \
    .getOrCreate()

23/04/08 18:08:57 WARN Utils: Your hostname, wedivv-H110M-S2V resolves to a loopback address: 127.0.1.1; using 192.168.1.44 instead (on interface wlp5s0)
23/04/08 18:08:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/08 18:08:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [36]:
cnaes = spark.read.csv('data/dados/cnaes/cnaes.csv',header=True, sep=';', encoding='ISO-8859-1', inferSchema=True)
cnaes.show(5)

+------+--------------------+
|  CNAE|      DESCRICAO_CNAE|
+------+--------------------+
|111301|    Cultivo de arroz|
|111302|    Cultivo de milho|
|111303|    Cultivo de trigo|
|111399|Cultivo de outros...|
|112101|Cultivo de algodã...|
+------+--------------------+
only showing top 5 rows



In [86]:
cnaes.printSchema()

root
 |-- CNAE: integer (nullable = true)
 |-- DESCRICAO_CNAE: string (nullable = true)



In [37]:
estabelecimentos = spark.read.csv('data/dados/estabelecimentos/*', header=True, sep=';', encoding='ISO-8859-1', inferSchema=True)
estabelecimentos.show(5)

+-----------+----------+-------+---------------------------+--------------------+------------------+-----------------------+-------------------------+--------------------------+----+---------------------+--------------+---------------+-----------------+--------------------+------+--------------------+--------------------+--------+---+---------+-----+--------+-----+--------+-------+--------+--------------------+-----------------+----------------------+
|CNPJ_BASICO|CNPJ_ORDEM|CNPJ_DV|IDENTIFICADOR_MATRIZ_FILIAL|       NOME_FANTASIA|SITUACAO_CADASTRAL|DATA_SITUACAO_CADASTRAL|MOTIVO_SITUACAO_CADASTRAL|NOME_DA_CIDADE_NO_EXTERIOR|PAIS|DATA_INICIO_ATIVIDADE|CNAE_PRINCIPAL|CNAE_SECUNDARIA|  TIPO_LOGRADOURO|          LOGRADOURO|NUMERO|         COMPLEMENTO|              BAIRRO|     CEP| UF|MUNICIPIO|DDD_1|   TEL_1|DDD_2|   TEL_2|DDD_FAX|     FAX|  CORREIO_ELETRONICO|SITUACAO_ESPECIAL|DATA_SITUACAO_ESPECIAL|
+-----------+----------+-------+---------------------------+--------------------+-------

In [32]:
estabelecimentos.count()

20996744

In [38]:
estabelecimentos.printSchema()

root
 |-- CNPJ_BASICO: integer (nullable = true)
 |-- CNPJ_ORDEM: integer (nullable = true)
 |-- CNPJ_DV: integer (nullable = true)
 |-- IDENTIFICADOR_MATRIZ_FILIAL: integer (nullable = true)
 |-- NOME_FANTASIA: string (nullable = true)
 |-- SITUACAO_CADASTRAL: integer (nullable = true)
 |-- DATA_SITUACAO_CADASTRAL: integer (nullable = true)
 |-- MOTIVO_SITUACAO_CADASTRAL: integer (nullable = true)
 |-- NOME_DA_CIDADE_NO_EXTERIOR: string (nullable = true)
 |-- PAIS: integer (nullable = true)
 |-- DATA_INICIO_ATIVIDADE: integer (nullable = true)
 |-- CNAE_PRINCIPAL: integer (nullable = true)
 |-- CNAE_SECUNDARIA: string (nullable = true)
 |-- TIPO_LOGRADOURO: string (nullable = true)
 |-- LOGRADOURO: string (nullable = true)
 |-- NUMERO: string (nullable = true)
 |-- COMPLEMENTO: string (nullable = true)
 |-- BAIRRO: string (nullable = true)
 |-- CEP: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- MUNICIPIO: integer (nullable = true)
 |-- DDD_1: string (nullable = true)

In [39]:
estabelecimentos.createOrReplaceTempView("estab_table")

spark.sql("SELECT COUNT(*) AS estabelecimentos FROM estab_table").show()

+----------------+
|estabelecimentos|
+----------------+
|        20996744|
+----------------+



In [40]:
estabelecimentos.write.parquet('data/dados/estabelecimentos/estabelecimentos.parquet')

In [45]:
spark.sql("SELECT COUNT(*) AS estab_sem_lograd FROM estab_table WHERE LOGRADOURO IS NULL").show()

+----------------+
|estab_sem_lograd|
+----------------+
|             828|
+----------------+



In [81]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

def is_avenida_fun(logra):
    if isinstance(logra, str) and logra.lower().startswith(('avenida')):
        return True
    return None

is_avenida = udf(is_avenida_fun, BooleanType())

spark.udf.register("is_avenida", is_avenida)

spark.sql("SELECT COUNT(*) as avenidas FROM estab_table WHERE is_avenida(LOGRADOURO) == True").show()

23/04/08 22:31:08 WARN SimpleFunctionRegistry: The function is_avenida replaced a previously registered function.


+--------+
|avenidas|
+--------+
|   52587|
+--------+



In [85]:
spark.sql("SELECT COUNT(DISTINCT CEP) FROM estab_table").show()

+-------------------+
|count(DISTINCT CEP)|
+-------------------+
|             889885|
+-------------------+



In [88]:
cnaes.createOrReplaceTempView('cnae_table')

spark.sql("SELECT COUNT(CNAE) FROM cnae_table").show()

+-----------+
|count(CNAE)|
+-----------+
|       1359|
+-----------+



In [98]:
complete = estabelecimentos.join(cnaes, estabelecimentos["CNAE_PRINCIPAL"] == cnaes["CNAE"], "inner")


In [99]:
complete.show(5)

+-----------+----------+-------+---------------------------+--------------------+------------------+-----------------------+-------------------------+--------------------------+----+---------------------+--------------+---------------+-----------------+--------------------+------+--------------------+--------------------+--------+---+---------+-----+--------+-----+--------+-------+--------+--------------------+-----------------+----------------------+-------+--------------------+
|CNPJ_BASICO|CNPJ_ORDEM|CNPJ_DV|IDENTIFICADOR_MATRIZ_FILIAL|       NOME_FANTASIA|SITUACAO_CADASTRAL|DATA_SITUACAO_CADASTRAL|MOTIVO_SITUACAO_CADASTRAL|NOME_DA_CIDADE_NO_EXTERIOR|PAIS|DATA_INICIO_ATIVIDADE|CNAE_PRINCIPAL|CNAE_SECUNDARIA|  TIPO_LOGRADOURO|          LOGRADOURO|NUMERO|         COMPLEMENTO|              BAIRRO|     CEP| UF|MUNICIPIO|DDD_1|   TEL_1|DDD_2|   TEL_2|DDD_FAX|     FAX|  CORREIO_ELETRONICO|SITUACAO_ESPECIAL|DATA_SITUACAO_ESPECIAL|   CNAE|      DESCRICAO_CNAE|
+-----------+----------+------

In [101]:

def cnae_desc(desc):
    if isinstance(desc, str) and desc.lower().startswith(('cultivo de')):
        return True
    return None

is_cultivo = udf(cnae_desc, BooleanType())

spark.udf.register("is_cultivo", is_cultivo)


<function __main__.cnae_desc(desc)>

In [104]:
complete.createOrReplaceTempView('full_table')

spark.sql("""
    SELECT COUNT(*)
    FROM full_table
    WHERE is_cultivo(DESCRICAO_CNAE) == True
""").show()

+--------+
|count(1)|
+--------+
|  200124|
+--------+



In [90]:
spark.sql("SELECT COUNT(*) from estab_table WHERE IDENTIFICADOR_MATRIZ_FILIAL = 2").show()

+--------+
|count(1)|
+--------+
| 1093082|
+--------+

